In [ ]:
'''mkdir Project_work
cd Project_work
mkdir Pdf_docs
copy the pdf files here
copy the provided 'sales_data.csv' here

--create virtual environment
python -m venv venv
cd  venv/Scripts
activate
(venv) C:\Users\AJ\Downloads\Project_work\venv>

--in virtual env install required packages
pip install pandas
python -m pip install --upgrade pip
pip install pypdf
pip install transformers
pip install numpy
pip install scipy
pip install openai
pip install langchain_community
pip install langchain-core
pip install langchain
'''

In [ ]:
#Project structure
#package to work with excel
#create df which contains from excel
#analysis = summary
#work with pdf documents >load > split > embedd > storing them in RAG layer
#langchain >
  #llm definition (myllm)
  #prompt template > scenario to form a prompt
  #prompt = PromptTemplate(template=scenario,input_variables=["analysis","question"])
#my_proj_chain = LLMChain(prompt=prompt,llm=myllm)
#my_proj_chain.run(analysis,question)
#question = 'based on my analysis on data, what strategies are recommeded for improvment of sales'
#my_proj_chain.run(analysis,question)

##implementation of seq chain to do the same thing as above in 2 different chains followed
#sequentially

#1st
#data_analysis_template = """analyze my {analysis} data  and provide a summary """
#a_prompt = PromptTemplate(template=data_analysis_template,input_variables=["analysis"])
#chain1 = LLMChain(prompt=a_prompt,llm=myllm,output='output')

#2nd
#recommendation_template = """based on {analysis} data  and provide a recommendaton to address the que """
#b_prompt = PromptTemplate(template=recommendation_template,input_variables=["analysis","question"])
#chain2 = LLMChain(prompt=b_prompt,llm=myllm,output='output2')

"""overall_chain = SequentialChain(
       chains = [chain1,chain2],
       input_variables=[],
       output_variables = [],
)

run the chain for a particular question"""
#------------

#Bulding RAG based layer
#once all data is chunked, embedded and pushed into FAISS vector store..
#llm = ChatOpenAI(temperature=0.7,model_name='gpt-4.1-mini')
"""retriever = vectorstore.as_retriever(search_type="similarity",sarch_kwags={"k": 2})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever =retriever,
    return_source_documents=True
)"""

#implementation of a tool like wikipedia search

#implemenation of usge of memory to benefit response generation

#integrate of evaluation
#answers to validate responses

In [ ]:
import pandas as pd
df = pd.read_csv('sales_data.csv')
df.describe

<bound method NDFrame.describe of             Date   Product Region  Sales  Customer_Age Customer_Gender  \
0       1/1/2022  Widget C  South    786            26            Male   
1       1/2/2022  Widget D   East    850            29            Male   
2       1/3/2022  Widget A  North    871            40          Female   
3       1/4/2022  Widget C  South    464            31            Male   
4       1/5/2022  Widget C  South    262            50          Female   
...          ...       ...    ...    ...           ...             ...   
2495  10/31/2028  Widget D  North    979            57            Male   
2496   11/1/2028  Widget D  South    858            30          Female   
2497   11/2/2028  Widget B   East    878            21          Female   
2498   11/3/2028  Widget C  South    862            63            Male   
2499   11/4/2028  Widget D   West    614            31            Male   

      Customer_Satisfaction  
0                  2.874407  
1                

In [4]:
df.columns

Index(['Date', 'Product', 'Region', 'Sales', 'Customer_Age', 'Customer_Gender',
       'Customer_Satisfaction'],
      dtype='object')

In [5]:
total_sales = df['Sales'].sum()
avg_sale = df['Sales'].mean()
median_sale = df['Sales'].median()
sales_std = df['Sales'].std()

In [10]:
df

,Date,Product,Region,Sales,Customer_Age,Customer_Gender,Customer_Satisfaction
0,1/1/2022,Widget C,South,786,26,Male,2.874407
1,1/2/2022,Widget D,East,850,29,Male,3.365205
2,1/3/2022,Widget A,North,871,40,Female,4.547364
3,1/4/2022,Widget C,South,464,31,Male,4.555420
4,1/5/2022,Widget C,South,262,50,Female,3.982935
...,...,...,...,...,...,...,...
2495,10/31/2028,Widget D,North,979,57,Male,3.525510
2496,11/1/2028,Widget D,South,858,30,Female,3.386064
2497,11/2/2028,Widget B,East,878,21,Female,2.272609
2498,11/3/2028,Widget C,South,862,63,Male,2.805692


In [6]:
df['Month'] = pd.to_datetime(df['Date']).dt.month

In [7]:
monthly_sales = df.groupby('Month', observed=False)['Sales'].sum().sort_values(ascending=False)

In [8]:
monthly_sales

Month
8     124264
5     123646
10    119428
1     118537
7     118493
4     117689
6     115935
3     115367
2     113135
9     111615
11    102775
12    102336
Name: Sales, dtype: int64

In [9]:
best_month = monthly_sales.index[0]

In [10]:
print(best_month)

8


In [11]:
def generate_advanced_data_summary(df):
    # Ensure 'Date' is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Sales Analysis
    total_sales = df['Sales'].sum()
    avg_sale = df['Sales'].mean()
    median_sale = df['Sales'].median()
    sales_std = df['Sales'].std()

    # Time-based Analysis
    df['Month'] = pd.to_datetime(df['Date']).dt.month
    monthly_sales = df.groupby('Month', observed=False)['Sales'].sum().sort_values(ascending=False)
    best_month = monthly_sales.index[0]
    worst_month = monthly_sales.index[-1]

    # Product Analysis
    product_sales = df.groupby('Product', observed=False)['Sales'].agg(['sum', 'count', 'mean'])
    top_product = product_sales['sum'].idxmax()
    most_sold_product = product_sales['count'].idxmax()

    # Regional Analysis
    region_sales = df.groupby('Region', observed=False)['Sales'].sum().sort_values(ascending=False)
    best_region = region_sales.index[0]
    worst_region = region_sales.index[-1]

    # Customer Analysis
    avg_satisfaction = df['Customer_Satisfaction'].mean()
    satisfaction_std = df['Customer_Satisfaction'].std()

    age_bins = [0, 25, 35, 45, 55, 100]
    age_labels = ['18-25', '26-35', '36-45', '46-55', '55+']
    df['Age_Group'] = pd.cut(df['Customer_Age'], bins=age_bins, labels=age_labels, right=False)
    age_group_sales = df.groupby('Age_Group', observed=False)['Sales'].mean().sort_values(ascending=False)
    best_age_group = age_group_sales.index[0]

    # Gender Analysis
    gender_sales = df.groupby('Customer_Gender', observed=False)['Sales'].mean()

    summary = f"""
    Advanced Sales Data Summary:

    Overall Sales Metrics:
    - Total Sales: ${total_sales:,.2f}
    - Average Sale: ${avg_sale:.2f}
    - Median Sale: ${median_sale:.2f}
    - Sales Standard Deviation: ${sales_std:.2f}

    Time-based Analysis:
    - Best Performing Month: {best_month}
    - Worst Performing Month: {worst_month}

    Product Analysis:
    - Top Selling Product (by value): {top_product}
    - Most Frequently Sold Product: {most_sold_product}

    Regional Performance:
    - Best Performing Region: {best_region}
    - Worst Performing Region: {worst_region}

    Customer Insights:
    - Average Customer Satisfaction: {avg_satisfaction:.2f}/5
    - Customer Satisfaction Standard Deviation: {satisfaction_std:.2f}
    - Best Performing Age Group: {best_age_group}
    - Gender-based Average Sales: Male=${gender_sales['Male']:.2f}, Female=${gender_sales['Female']:.2f}


    Key Observations:
    1. The sales data shows significant variability with a standard deviation of ${sales_std:.2f}.
    2. The {best_age_group} age group shows the highest average sales.
    3. Regional performance varies significantly, with {best_region} outperforming {worst_region}.
    4. The most valuable product ({top_product}) differs from the most frequently sold product ({most_sold_product}), suggesting potential for targeted marketing strategies.
    """

    return summary

In [12]:
print(generate_advanced_data_summary(df))


    Advanced Sales Data Summary:

    Overall Sales Metrics:
    - Total Sales: $1,383,220.00
    - Average Sale: $553.29
    - Median Sale: $552.50
    - Sales Standard Deviation: $260.10

    Time-based Analysis:
    - Best Performing Month: 8
    - Worst Performing Month: 12

    Product Analysis:
    - Top Selling Product (by value): Widget A
    - Most Frequently Sold Product: Widget A

    Regional Performance:
    - Best Performing Region: West
    - Worst Performing Region: East

    Customer Insights:
    - Average Customer Satisfaction: 3.03/5
    - Customer Satisfaction Standard Deviation: 1.16
    - Best Performing Age Group: 18-25
    - Gender-based Average Sales: Male=$547.56, Female=$558.96


    Key Observations:
    1. The sales data shows significant variability with a standard deviation of $260.10.
    2. The 18-25 age group shows the highest average sales.
    3. Regional performance varies significantly, with West outperforming East.
    4. The most valuable produ

In [13]:
summary = generate_advanced_data_summary(df)

In [14]:
print(summary)


    Advanced Sales Data Summary:

    Overall Sales Metrics:
    - Total Sales: $1,383,220.00
    - Average Sale: $553.29
    - Median Sale: $552.50
    - Sales Standard Deviation: $260.10

    Time-based Analysis:
    - Best Performing Month: 8
    - Worst Performing Month: 12

    Product Analysis:
    - Top Selling Product (by value): Widget A
    - Most Frequently Sold Product: Widget A

    Regional Performance:
    - Best Performing Region: West
    - Worst Performing Region: East

    Customer Insights:
    - Average Customer Satisfaction: 3.03/5
    - Customer Satisfaction Standard Deviation: 1.16
    - Best Performing Age Group: 18-25
    - Gender-based Average Sales: Male=$547.56, Female=$558.96


    Key Observations:
    1. The sales data shows significant variability with a standard deviation of $260.10.
    2. The 18-25 age group shows the highest average sales.
    3. Regional performance varies significantly, with West outperforming East.
    4. The most valuable produ

In [20]:
from langchain import PromptTemplate, LLMChain

In [22]:
from langchain_openai import AzureChatOpenAI

In [27]:
#from langchain_openai import OpenAI

In [ ]:
llm = AzureChatOpenAI(
    azure_deployment="gpt-4.1-mini",         # deployment name from Azure portal
    azure_endpoint="https://singalajay-7416-resource.cognitiveservices.azure.com/",           # your endpoint
    api_version="2023-12-01-preview",
    api_key="mykey",
    temperature=0.0
)

In [29]:
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001AEE3E58E50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001AEE4399300>, root_client=<openai.lib.azure.AzureOpenAI object at 0x000001AEE3456860>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x000001AEE3E59960>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), disabled_params={'parallel_tool_calls': None}, azure_endpoint='https://singhalkajay-7416-resource.cognitiveservices.azure.com/', deployment_name='gpt-4.1-mini', openai_api_version='2023-12-01-preview', openai_api_type='azure')

In [30]:
summary

'\n    Advanced Sales Data Summary:\n\n    Overall Sales Metrics:\n    - Total Sales: $1,383,220.00\n    - Average Sale: $553.29\n    - Median Sale: $552.50\n    - Sales Standard Deviation: $260.10\n\n    Time-based Analysis:\n    - Best Performing Month: 8\n    - Worst Performing Month: 12\n\n    Product Analysis:\n    - Top Selling Product (by value): Widget A\n    - Most Frequently Sold Product: Widget A\n\n    Regional Performance:\n    - Best Performing Region: West\n    - Worst Performing Region: East\n\n    Customer Insights:\n    - Average Customer Satisfaction: 3.03/5\n    - Customer Satisfaction Standard Deviation: 1.16\n    - Best Performing Age Group: 18-25\n    - Gender-based Average Sales: Male=$547.56, Female=$558.96\n\n\n    Key Observations:\n    1. The sales data shows significant variability with a standard deviation of $260.10.\n    2. The 18-25 age group shows the highest average sales.\n    3. Regional performance varies significantly, with West outperforming East

In [31]:
scenario_template = """
You are an AI sales analyst. Use the summary data which we have analyzed, provide some in-depth analysis and some recommendation based on 
data provided. Be specific to the data points.
{summary}
Question: {question}

Detaled Analysis & recommendation by AI:
"""

In [37]:
prompt = PromptTemplate(template=scenario_template,input_variables=["summary","question"])

In [52]:
#older version
llm_chain = LLMChain(prompt=prompt,llm=llm)

In [53]:
#test
#llm_chain.run(summary=summary,question="based on this data, what are our areas of improvement")

In [54]:
question = "based on this data, what are our areas of improvement"
def gen_insight(summary,question):
    return llm_chain.run(summary=summary,question=question)

In [55]:
#newer version
#chain = prompt | llm
#question = "based on this data, what are our areas of improvement"
#def gen_insight(summary, question):
#    return chain.invoke({"summary": summary, "question": question}).content

In [56]:
insight = gen_insight(summary,question)
print(insight)

Certainly! Below is a detailed analysis and specific recommendations based on the advanced sales data summary provided:

---

### Detailed Analysis

#### 1. **Sales Variability and Consistency**
- **Observation:** The sales standard deviation is $260.10, which is relatively high compared to the average sale of $553.29. This indicates significant variability in transaction sizes.
- **Implication:** High variability can signal inconsistent customer purchasing behavior or a wide range of product pricing. This inconsistency may complicate forecasting and inventory management.

#### 2. **Time-based Performance**
- **Observation:** August (Month 8) is the best performing month, while December (Month 12) is the worst.
- **Implication:** There may be seasonal factors or promotional activities driving sales in August. Conversely, December’s poor performance is unusual given it is typically a high-sales month in many industries, suggesting missed opportunities or external factors negatively impa

In [ ]:
#from langchain import SequentialChain
##implementation of seq chain to do the same thing as above in 2 different chains followed
#sequentially

#1st
#data_analysis_template = """analyze my {analysis} data  and provide a summary """
#a_prompt = PromptTemplate(template=data_analysis_template,input_variables=["analysis"])
#chain1 = LLMChain(prompt=a_prompt,llm=myllm,output='output')

#2nd
#recommendation_template = """based on {analysis} data  and provide a recommendaton to address the question: {question} """
#b_prompt = PromptTemplate(template=recommendation_template,input_variables=["analysis","question"])
#chain2 = LLMChain(prompt=b_prompt,llm=myllm,output='output2')

'''overall_chain = SequentialChain(
       chains = [chain1,chain2],
       input_variables=['summary','question'],
       output_variables = ['analysis','recommendation'],
)'''

In [57]:
'''def gen_insight(question):
    result = overall_chain.run({"summary": summary,"question": question})
    return f"Analysis:\n{result['analysis']\n\nRecommendation:\n{result['recommendation']}"'''

'def gen_insight(question):\n    result = overall_chain.run({"summary": summary,"question": question})\n    return f"Analysis:\n{result[\'analysis\']\n\nRecommendation:\n{result[\'recommendation\']}"'

In [58]:
'''question = "how can we improve in a region"
insight = gen_insight(question)
print(insight)'''

'question = "how can we improve in a region"\ninsight = gen_insight(question)\nprint(insight)'

In [59]:
from langchain.document_loaders import PyPDFLoader

In [61]:
!pip install pypdf


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [62]:
import pypdf

In [63]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [64]:
import os
os.listdir('PDF Folder')

['AI business model innovation.pdf',
 'BI approaches.pdf',
 'Time-Series-Data-Prediction-using-IoT-and-Machine-Le_2020_Procedia-Computer-.pdf',
 'Walmarts sales data analysis.pdf']

In [65]:
pdf_folder = 'PDF Folder'

In [66]:
documents = []
for file in os.listdir(pdf_folder):
    if file.endswith('.pdf'):
        loader = PyPDFLoader(os.path.join(pdf_folder, file))
        documents.extend(loader.load())

In [68]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap= 200)

In [74]:
texts = text_splitter.split_documents(documents)

In [75]:
#optional
#save your texts as .pkl

In [76]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model)

C:\Users\Ajay\AppData\Local\Temp\ipykernel_16088\3834499437.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model)


In [77]:
!pip install faiss-cpu


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [78]:
from langchain.vectorstores import FAISS

In [79]:
from langchain.chains import RetrievalQA

In [80]:
#from langchain.utilities import WikipediaAPIWrapper
#from datetime import datetime

In [81]:
vectorstore = FAISS.from_documents(texts, embeddings)

In [83]:
#Testing
'''
all_docs = list(vectorstore.docstore._dict.values())

for i, doc in enumerate(all_docs):
    print(f"Document {i+1}:\n{doc.page_content}\n")'''

'\nall_docs = list(vectorstore.docstore._dict.values())\n\nfor i, doc in enumerate(all_docs):\n    print(f"Document {i+1}:\n{doc.page_content}\n")'

In [84]:
retriever = vectorstore.as_retriever(search_type="similarity", k=4)

In [86]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [87]:
#define a function to use wiki search

In [88]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

C:\Users\Ajay\AppData\Local\Temp\ipykernel_16088\3326585183.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
C:\Users\Ajay\AppData\Local\Temp\ipykernel_16088\3326585183.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(


In [89]:
question = 'what are top selling products'

In [90]:
context = f"my summary data is:\n{summary}\nQuestion:{question}"

In [91]:
result = qa_chain({"query": context})

C:\Users\Ajay\AppData\Local\Temp\ipykernel_16088\431315255.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": context})


In [93]:
#result

In [94]:
#get wiki content related to question

In [96]:
conversation.predict(input=f"my summary data is:\n{summary}\nQuestion:{question}")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: my summary data is:

    Advanced Sales Data Summary:

    Overall Sales Metrics:
    - Total Sales: $1,383,220.00
    - Average Sale: $553.29
    - Median Sale: $552.50
    - Sales Standard Deviation: $260.10

    Time-based Analysis:
    - Best Performing Month: 8
    - Worst Performing Month: 12

    Product Analysis:
    - Top Selling Product (by value): Widget A
    - Most Frequently Sold Product: Widget A

    Regional Performance:
    - Best Performing Region: West
    - Worst Performing Region: East

    Customer Insights:
    - Average Customer Satisfaction: 3.03/5
    - Customer Satisfaction Standard Deviation: 1.16
    - Best Performing Age Group: 18

'The top selling product by value in your sales data is Widget A. Additionally, Widget A is also the most frequently sold product. This means that Widget A not only generates the highest total sales revenue but is also sold more often than any other product. This dual status highlights Widget A as a key driver of your sales performance and suggests it could be a focal point for marketing and sales strategies. If you want, I can help you explore targeted marketing approaches or analyze other products in more detail!'

In [97]:
def insights_based_on_memory(question):
    return conversation.predict(input=f"my summary data is:\n{summary}\nQuestion:{question}")

In [98]:
insights_based_on_memory("which region is performing better and why")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: my summary data is:

    Advanced Sales Data Summary:

    Overall Sales Metrics:
    - Total Sales: $1,383,220.00
    - Average Sale: $553.29
    - Median Sale: $552.50
    - Sales Standard Deviation: $260.10

    Time-based Analysis:
    - Best Performing Month: 8
    - Worst Performing Month: 12

    Product Analysis:
    - Top Selling Product (by value): Widget A
    - Most Frequently Sold Product: Widget A

    Regional Performance:
    - Best Performing Region: West
    - Worst Performing Region: East

    Customer Insights:
    - Average Customer Satisfaction: 3.03/5
    - Customer Satisfaction Standard Deviation: 1.16
    - Best Performing Age Group: 18-

"The best performing region in your sales data is the West, while the worst performing region is the East. Although the summary doesn't provide specific reasons for this regional performance difference, several factors could contribute:\n\n1. **Market Demand:** The West region might have higher demand for your products, possibly due to demographic factors, economic conditions, or consumer preferences aligning better with your offerings.\n\n2. **Sales and Marketing Efforts:** There could be stronger or more effective sales and marketing campaigns in the West, leading to higher sales volumes and values.\n\n3. **Distribution and Availability:** Better distribution networks or product availability in the West might make it easier for customers to purchase your products.\n\n4. **Customer Satisfaction and Engagement:** While the overall average customer satisfaction is 3.03/5, regional variations might exist, with the West possibly having higher satisfaction, encouraging repeat purchases.\n\

In [ ]:
#QAEvalChain